# Getting expert step by step

## Counting different elements

## Internal counters

add the doc for this and let the user test

## Multiple steps

mrjob can be configured to run different steps

for each step you can specify which part has to be executed
and the method to use within the class you wrote


In [2]:
# -*- coding: utf-8 -*-
""" MapReduce easily with Python """

from mrjob.job import MRJob
from mrjob.step import MRStep

class job(MRJob):
    def mapper(self, _, line):
        pass
    def another_method(self, key, line):
        pass
    def reducer(self, key, line):
        pass
    def steps(self):
        return [ 
            MRStep(mapper=self.mapper, reducer=self.reducer),
            MRStep(mapper=self.another_method),
        ]

if __name__ == "__main__":
    job.run()

Usage: __main__.py [options] [input files]

__main__.py: error: no such option: -f


SystemExit: 2

To exit: use 'exit', 'quit', or Ctrl-D.


Another reducer...

In [ ]:
%%writefile $myscript

from mrjob.job import MRJob
import re

WORD_RE = re.compile(r"[\w']+")

class MRMostUsedWord(MRJob):

    def mapper_get_words(self, _, line):
        # yield each word in the line
        for word in WORD_RE.findall(line):
            yield (word.lower(), 1)

    def combiner_count_words(self, word, counts):
        # optimization: sum the words we've seen so far
        yield (word, sum(counts))

    def reducer_count_words(self, word, counts):
        # send all (num_occurrences, word) pairs to the same reducer.
        # num_occurrences is so we can easily use Python's max() function.
        yield None, (sum(counts), word)

    # discard the key; it is just None
    def reducer_find_max_word(self, _, word_count_pairs):
        # each item of word_count_pairs is (count, word),
        # so yielding one results in key=counts, value=word
        yield max(word_count_pairs)

    def steps(self):
        return [
            self.mr(mapper=self.mapper_get_words,
                    combiner=self.combiner_count_words,
                    reducer=self.reducer_count_words),
            self.mr(reducer=self.reducer_find_max_word)
        ]


if __name__ == '__main__':
    MRMostUsedWord.run()

vowels extra

In [ ]:
# -*- coding: utf-8 -*-

from mrjob.job import MRJob
from mrjob.step import MRStep


class job(MRJob):

    def mapper(self, _, line):
        for char in line.strip().lower():
            if char in 'aeiou':
                yield char, 1

    def reducer(self, key, values):
        mysum = sum(values)
        self.increment_counter("vowels", key, mysum)
        # Move sum and key as values
        yield None, [mysum, key]

    def find_min_max(self, _, values):
        # Avoid consuming the generator more than once
        data = list(values)
        # Use available data to make different computations
        yield "vowel_min_counter", min(data)
        yield "vowel_max_counter", max(data)

    def steps(self):
        return [
            MRStep(mapper=self.mapper, reducer=self.reducer),
            MRStep(reducer=self.find_min_max),
        ]

if __name__ == "__main__":
    job.run()


## Counting DNA positions

In [ ]:
%%bash

# Download compressed NGS data from a link
wget "http://bit.ly/ngs_sample_data" -O /tmp/ngs.sam.bz2
# Decompress the file
bunzip2 /tmp/ngs.sam.bz2

echo "Completed"

# End of Chapter

In [ ]:
%load_ext watermark
%watermark -a "Paolo D." -d -v -m -p jupyter